# Dia 2 - Limpeza e Enriquecimento de Dados

Neste segundo desafio do #7DaysOfCode, vamos manipular e preparar os dados da biblioteca. O foco será a limpeza da base e a criação de uma nova coluna classificando as obras de acordo com a CDU (Classificação Decimal Universal).

Vamos nessa!

### Importando bibliotecas

In [30]:
import pandas as pd

### Importação do Dataset

Importamos o dataset gerado no Desafio 01, que está salvo como `dataset_completo.csv`.


In [39]:
# Carregando o DataFrame
df = pd.read_csv('../desafio02/dataset_completo.csv')

In [40]:
df.head(3)

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao,registro_sistema
0,709684,L095049,NaN,2010-01-04 07:44:10.721,2010-01-05 16:26:12.662,2.008023e+09,ALUNO DE GRADUAÇÃO,13259.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,630.0,4225.0
1,709685,L167050,NaN,2010-01-04 07:44:10.750,2010-01-12 07:34:13.934,2.008023e+09,ALUNO DE GRADUAÇÃO,70865.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,647.0,25009.0
2,709686,2006017618,2010-01-26 08:07:01.738,2010-01-04 08:08:44.081,2010-02-25 07:36:25.800,2.008112e+09,ALUNO DE PÓS-GRADUAÇÃO,195347.0,Acervo Circulante,Biblioteca Setorial Prof. Rodolfo Helinski - E...,REGULAR,640.0,75019.0


### Remoção de Coluna Irrelevante

A coluna `registro_sistema` não traz informações úteis para a análise e foi removida.


In [41]:
# Removendo a coluna 'registro_sistema'

df = df.drop(columns = ['registro_sistema'])
df.head(2)

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao
0,709684,L095049,NaN,2010-01-04 07:44:10.721,2010-01-05 16:26:12.662,2.008023e+09,ALUNO DE GRADUAÇÃO,13259.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,630.0
1,709685,L167050,NaN,2010-01-04 07:44:10.750,2010-01-12 07:34:13.934,2.008023e+09,ALUNO DE GRADUAÇÃO,70865.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,647.0


### Conversão de Tipos

Transformamos a coluna `matricula_ou_siape` em string para melhor leitura e compatibilidade.


In [34]:
# Convertendo a coluna 'matricula_ou_siape' em uma string

df['matricula_ou_siape'] = df['matricula_ou_siape'].astype(str)
df.head(2)

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao
0,709684,L095049,NaN,2010-01-04 07:44:10.721,2010-01-05 16:26:12.662,2008023265.0,ALUNO DE GRADUAÇÃO,13259.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,630.0
1,709685,L167050,NaN,2010-01-04 07:44:10.750,2010-01-12 07:34:13.934,2008023265.0,ALUNO DE GRADUAÇÃO,70865.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,647.0


### Verificação da Coluna "localizacao"

Aqui fazemos uma checagem da coluna `localizacao`, que trará a base para a classificação das obras.


In [42]:
df['localizacao'].head(10)

0    630.0
1    647.0
2    640.0
3    609.0
4    686.0
5    698.0
6    698.0
7    645.0
8    658.0
9    697.0
Name: localizacao, dtype: float64

In [43]:
df['localizacao'].dtype

dtype('float64')

### Tratamento e Conversão da Coluna "localizacao"

Preenchemos valores nulos com 0, forçamos o tipo `float`, e depois `int`, para padronizar a coluna antes da classificação.


In [44]:
# Convertendo a coluna 'localizacao' do tipo float para int

df['localizacao'] = df['localizacao'].fillna(0).astype(float).astype(int)
df['localizacao'].dtype

dtype('int64')

In [45]:
df['localizacao'].head(10)

0    630
1    647
2    640
3    609
4    686
5    698
6    698
7    645
8    658
9    697
Name: localizacao, dtype: int64

### Função de Classificação CDU

Criamos uma função que mapeia os valores da coluna `localizacao` de acordo com a CDU, categorizando os livros por área temática.


In [46]:
def classificar_cdu(valor):
    try:
        valor = int(str(valor).strip())  # Garantir conversão para inteiro
        if 0 <= valor <= 99:
            return 'Generalidades. Ciência e conhecimento'
        elif 100 <= valor <= 199:
            return 'Filosofia e psicologia'
        elif 200 <= valor <= 299:
            return 'Religião'
        elif 300 <= valor <= 399:
            return 'Ciências sociais'
        elif 400 <= valor <= 499:
            return 'Classe vaga'
        elif 500 <= valor <= 599:
            return 'Matemática e ciências naturais'
        elif 600 <= valor <= 699:
            return 'Ciências aplicadas'
        elif 700 <= valor <= 799:
            return 'Belas artes'
        elif 800 <= valor <= 899:
            return 'Linguagem. Língua. Linguística'
        elif 900 <= valor <= 999:
            return 'Geografia. Biografia. História'
        else:
            return 'Código fora da faixa CDU'
    except:
        return 'Valor inválido'



### Aplicando a Classificação

Com a função pronta, aplicamos sobre a coluna `localizacao` para gerar a nova coluna `classe_cdu`.


In [47]:
df['classe_cdu'] = df['localizacao'].apply(classificar_cdu)
df.head(3)

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao,classe_cdu
0,709684,L095049,NaN,2010-01-04 07:44:10.721,2010-01-05 16:26:12.662,2.008023e+09,ALUNO DE GRADUAÇÃO,13259.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,630,Ciências aplicadas
1,709685,L167050,NaN,2010-01-04 07:44:10.750,2010-01-12 07:34:13.934,2.008023e+09,ALUNO DE GRADUAÇÃO,70865.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,647,Ciências aplicadas
2,709686,2006017618,2010-01-26 08:07:01.738,2010-01-04 08:08:44.081,2010-02-25 07:36:25.800,2.008112e+09,ALUNO DE PÓS-GRADUAÇÃO,195347.0,Acervo Circulante,Biblioteca Setorial Prof. Rodolfo Helinski - E...,REGULAR,640,Ciências aplicadas


### Exportação dos Dados

Exportamos o novo DataFrame para o desafio03 com os dados limpos e enriquecidos com a classificação temática.


In [48]:
# Exportando o DataFrame para csv para uso no desafio03

df.to_csv('../desafio03/dataset_completo.csv', index = False)